In [ ]:
import pylab as plt
import lsst.afw.display as afwDisplay


In [ ]:
def displayExposure(exposure,title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    afwDisplay.setImageColormap(cmap='gray')
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(exposure.getImage())
    plt.title(title)
    plt.gca().axis('off')
    return afw_display

def displayImage(image,title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    afw_display.scale('asinh', 'zscale')
    #afw_display.scale('linear', min=-5, max=10)
    afw_display.setImageColormap(cmap='plasma')
    afw_display.mtv(image)
    plt.title(title)
    plt.gca().axis('off')
    return afw_display
    
def displayImageGhosts(image, zmin=0, zmax=5000, title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    #afw_display.scale('asinh', 'zscale')
    afw_display.scale('linear', min=zmin, max=zmax)
    afw_display.setImageColormap(cmap='plasma')
    afw_display.mtv(image)
    plt.title(title)
    plt.gca().axis('off')
    return afw_display    

In [ ]:
# Define butler
from lsst.daf.butler import Butler

repo = 'embargo_new'
collection = 'LSSTComCam/nightlyValidation'
butler = Butler(repo,collections=collection)
registry = butler.registry


In [ ]:
def queryExposures(expo="2024102400128"):
    dataset_refs = butler.query_datasets("postISRCCD", collections="LSSTComCam/nightlyValidation",
                                         where=f"instrument='LSSTComCam' AND exposure={expo}")
    exposures = [butler.get(dataset_ref) for dataset_ref in dataset_refs]
    return exposures

def make_mosaic(exposures, binning=4):
    from lsst.pipe.tasks.visualizeVisit import (
        VisualizeBinExpConfig,
        VisualizeBinExpTask,
        VisualizeMosaicExpConfig,
        VisualizeMosaicExpTask,
    )
    camera = butler.get("camera", collections="LSSTComCam/defaults", instrument="LSSTComCam")
    
    visualizeBinExpConfig = VisualizeBinExpConfig()
    visualizeBinExpConfig.binning = binning
    visualizeBinExpTask = VisualizeBinExpTask(config=visualizeBinExpConfig)
    exposures_binned = [visualizeBinExpTask.run(inputExp = exposure, camera=camera).outputExp for exposure in exposures]
    
    visualizeMosaicExpConfig = VisualizeMosaicExpConfig()
    visualizeMosaicExpConfig.binning = binning
    visualizeMosaicExpTask = VisualizeMosaicExpTask(config=visualizeMosaicExpConfig)
    
    mosaic_full = visualizeMosaicExpTask.run(inputExps=exposures_binned, camera=camera)
    mosaic = mosaic_full.outputData
    return mosaic, mosaic_full

def show_ghosts(exp_id="2024102400126", binning=4, zmin=2230, zmax=2330):
    expos = queryExposures(exp_id)
    mosaic, mosaic_full = make_mosaic(expos, binning)
    displayImageGhosts(mosaic, zmin=zmin, zmax=zmax)
    return mosaic, mosaic_full, expos

In [ ]:
mosaic, mosaic_full, expos = show_ghosts("2024102400126", binning=4, zmin=2230, zmax=2330)

In [ ]:
type(mosaic_full)
mosaic_full.getDict().values()


In [ ]:
w = expos[0].getWcs()
type(expos[0])

In [ ]:
mosaic.getWcs()

In [ ]:
ghost_367, expos_367 = show_ghosts("2024112200367", binning=8, zmin=500, zmax=2000)

In [ ]:
displayImageGhosts(ghost, zmin=820, zmax=900)

In [ ]:
afwDisplay.setDefaultBackend('matplotlib') 
fig = plt.figure(figsize=(10,10))
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
#afw_display.scale('linear', min=zmin, max=zmax)
afw_display.setImageColormap(cmap='grey')
afw_display.mtv(ghost_367)
plt.title("MC_2024112200367")
plt.gca().axis('off')


In [ ]:
ghost_292, expos_292 = show_ghosts("2024111100292", binning=8, zmin=500, zmax=2000)

In [ ]:
afwDisplay.setDefaultBackend('matplotlib') 
fig = plt.figure(figsize=(10,10))
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
#afw_display.scale('linear', min=zmin, max=zmax)
afw_display.setImageColormap(cmap='grey')
afw_display.mtv(ghost_292)
plt.title("MC_2024112200367")
plt.gca().axis('off')

In [ ]:
type(mosaic)

In [ ]:
mosaic.getBBox()

In [ ]:
expos_292[0].visitInfo.getBoresightRaDec()

In [ ]:
expos_292[0].getMetadata()["RA"]

In [ ]:
ghost_292.writeFits("ghost_292.fits", expos_292[0].getMetadata(), "w")

In [ ]:
ghost_292.getDimensions()

In [ ]:
ghost_367.writeFits("ghost_367.fits", expos_367[0].getMetadata(), "w")